In [1]:
from data.BikeDataImporter import BikeDataImporter
from data.DataPreparing import DataPreparing

import lightgbm as lgb
import pandas as pd
import numpy as np
import holidays
from suntime import Sun, SunTimeException
from datetime import datetime
import optuna
from sklearn.metrics import mean_pinball_loss
from sklearn.model_selection import TimeSeriesSplit
from sktime.split import ExpandingSlidingWindowSplitter
from sklearn.metrics import d2_pinball_score
from statsmodels.tsa.seasonal import MSTL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib import pyplot as plt
import os
from pathlib import Path
from sklearn.linear_model import QuantileRegressor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from statsforecast import StatsForecast
from statsforecast.models import MSTL
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib as mpl
import plotly.graph_objects as go

c:\Users\simon\.conda\envs\ptsc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
plt.style.use("stylesheet.mplstyle")

In [ ]:
def get_bike_and_weather_data():
    """Get the bike data and the weather data as a single dataframe.

    Returns:
        data_w_weather (pd.DataFrame): The bike data with the weather data.
    """
    bike_data = get_bike_data()
    # Give the last date of the bike data
    weather_data_hourly = get_weather_data_hourly()
    weather_data_daily = get_weather_data_daily(weather_data_hourly)
    data_w_weather = combine_data_weather(bike_data, weather_data_daily)
    # Filter the data to the last date of the bike data

    return data_w_weather

def get_bike_data():
    """Call the BikeDataImporter to get the bike data.

    Args:
        start_date (str): The start date for the data.

    Returns:
        data (pd.Dataframe): Bike counts for each day.
    """
    bike_data = BikeDataImporter()
    data = bike_data.get_bike_data()
    return data

def get_weather_data_hourly():
    """Get the weather data for Karlsruhe from the historical and forecast data on hourly basis.

    Returns:
        weather_data (pd.Dataframe): Hourly historical and forecast weather data.
    """
    todays_date = pd.Timestamp.now().strftime("%Y%m%d")
    # Load the historical and the forecast weather data
    history_data_path = os.path.join(os.path.dirname(os.getcwd()), "data", "weather", "history", "karlsruhe", f"history_karlsruhe_{todays_date}.csv")
    forecast_data_path = os.path.join(os.path.dirname(os.getcwd()), "data", "weather", "forecasts", "karlsruhe", f"forecast_karlsruhe_{todays_date}.csv")
    
    # Read in the historical and forecast data
    history_data = pd.read_csv(history_data_path)
    forecast_data = pd.read_csv(forecast_data_path)

    # Set the index to the date
    history_data.set_index('date', inplace=True)
    forecast_data.set_index('date', inplace=True)

    # Convert the index to datetime
    history_data.index = pd.to_datetime(history_data.index)
    forecast_data.index = pd.to_datetime(forecast_data.index)

    # Convert the index to datetime
    history_data.index = history_data.index.tz_convert('UTC')
    forecast_data.index = forecast_data.index.tz_convert('UTC')

    # Convert the index to European timezone (e.g., Europe/Paris)
    # history_data.index = history_data.index.tz_convert('Europe/Paris')
    # forecast_data.index = forecast_data.index.tz_convert('Europe/Paris')

    # The history data is not complete, so we need to filter it
    max_hisory_date = pd.Timestamp.now(tz='UTC') - pd.Timedelta(days=2)
    history_data_filtered = history_data.loc[(history_data.index < max_hisory_date )].copy()
    forecast_data_filtered = forecast_data.loc[(forecast_data.index >= max_hisory_date)].copy()

    # Merge the historical and forecast data
    weather_data = pd.concat([history_data_filtered, forecast_data_filtered], axis=0)

    return weather_data.iloc[1:]

def get_weather_data_daily(weather_data_hourly):
    """Resample the hourly weather data to daily data.

    Args:
        weather_data_hourly (pd.Dataframe): Hourly historical and forecast weather data.

    Returns:
        weather_data_daily (pd.Dataframe): Daily historical and forecast weather data.
    """
    weather_data_daily = weather_data_hourly.resample('D').mean() # Todo: Check if we need to resample differently
    return weather_data_daily

def combine_data_weather(bike_data, weather_data):
    """Merge the bike data with the weather data.

    Args:
        bike_data (pd.Dataframe): Daily bike data.
        weather_data (pd.Dataframe): Daily weather data.

    Returns:
        bike_and_weather_data (pd.Dataframe): bike_and_weather_data
    """
    bike_and_weather_data = pd.merge(bike_data, weather_data, left_index=True, right_index=True, how='left')
    return bike_and_weather_data
  
def create_features(data):
    """Create the features based on the data.

    Args:
        data (_type_): _description_

    Returns:
        _type_: _description_
    """
    data = data.copy()

    # Create time features
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['quarter'] = data.index.quarter
    data['weekday'] = data.index.weekday

    # Create lag features
    target_map = data['bike_count'].to_dict()
    # data['lag_1'] = (data.index - pd.Timedelta(days=1)).map(target_map) # Todo: Change to iterativ approach so that we can use these features as well.
    # data['lag_2'] = (data.index - pd.Timedelta(days=2)).map(target_map)
    # data['lag_3'] = (data.index - pd.Timedelta(days=3)).map(target_map)
    # data['lag_4'] = (data.index - pd.Timedelta(days=4)).map(target_map)
    data['lag_5'] = (data.index - pd.Timedelta(days=7)).map(target_map)
    data['lag_6'] = (data.index - pd.Timedelta(days=14)).map(target_map)
    data['lag_7'] = (data.index - pd.Timedelta(days=20)).map(target_map)
    data['lag_8'] = (data.index - pd.Timedelta(days=28)).map(target_map)
    data['lag_9'] = (data.index - pd.Timedelta(days=50)).map(target_map)

    # Create holiday feature
    data["is_holiday"] = 0
    bw_holidays = holidays.country_holidays("DE", subdiv="BW")
    for i, date in enumerate(data.index):
        if date in bw_holidays:
            data.at[date, "is_holiday"] = 1

    # Create corona feature
    lockdown1_start = pd.Timestamp('2020-03-16', tz='UTC')
    lockdown1_end = pd.Timestamp('2020-05-11', tz='UTC')
    easing_start = pd.Timestamp('2020-06-01', tz='UTC')
    easing_end = pd.Timestamp('2020-10-30', tz='UTC')
    lockdown2_start = pd.Timestamp('2020-11-02', tz='UTC')
    lockdown2_end = pd.Timestamp('2020-02-14', tz='UTC')
    vaccionation_start = pd.Timestamp('2021-03-01', tz='UTC')
    vaccionation_end = pd.Timestamp('2021-06-30', tz='UTC')

    data.loc[(data.index < lockdown1_start), 'corona_phase'] = 4 # Pre-Pandemic
    data.loc[(data.index >= lockdown1_start) & (data.index <= lockdown1_end), 'corona_phase'] = 0 # First Lockdown
    data.loc[(data.index >= easing_start) & (data.index <= easing_end), 'corona_phase'] = 5 # Easing
    data.loc[(data.index >= lockdown2_start) & (data.index <= lockdown2_end), 'corona_phase'] = 1 # Second Lockdown
    data.loc[(data.index >= vaccionation_start) & (data.index <= vaccionation_end), 'corona_phase'] = 2 # Vaccination Rollout
    data.loc[(data.index > vaccionation_end), 'corona_phase'] = 3 # Post-Vaccination


    # Create rolling features
    # data['rolling_mean_7'] = (data['bike_count'].rolling(window=7).mean())
    # data['rolling_mean_30'] = (data['bike_count'].rolling(window=30).mean())
    # data["rolling_std_7"] = data["bike_count"].rolling(window=7).std()
    # data["rolling_std_30"] = data["bike_count"].rolling(window=30).std()
    # data["diff_prev_day"] = data["bike_count"].diff()

    return data

data_preparer = DataPreparing()
data = data_preparer.get_bike_and_weather_data()
data = data_preparer.create_features(data) # TTODO Still add here the iterative approach for the lag features

In [ ]:
# Plot the correlation of the features with the target bike_count
correlation = data.corr()
correlation['bike_count'].sort_values(ascending=False).plot(kind='bar', figsize=(20, 10))

In [ ]:
# Plot the peasron correlation matrix
correlation = data.corr()

# Select from the data just some specific columns
data_selected = data[
    [
        "bike_count",
        "temperature_2m",
        "precipitation",
        "rain",
        "snowfall",
        "snow_depth",
        "month",
        "quarter",
        "weekday",
        "lag_5",
        "lag_6",
        "lag_7",
        "lag_8",
        "lag_9",
        "is_holiday",
        "corona_phase",
    ]
]
plt.figure(figsize=(20, 10))
correlation = data_selected.corr()
sns.heatmap(correlation, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()


In [ ]:
# Assuming `data['bike_count']` contains the target variable
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# Plot ACF
plot_acf(data['bike_count'], lags=30, ax=axes[0])
axes[0].set_title("ACF", fontsize=14)
axes[0].set_xlabel("Lags", fontsize=12)
axes[0].set_ylabel("ACF", fontsize=12)

# Plot PACF
plot_pacf(data['bike_count'], lags=30, ax=axes[1], method='ywm')
axes[1].set_title("PACF", fontsize=14)
axes[1].set_xlabel(r"Lags", fontsize=12)
axes[1].set_ylabel("PACF", fontsize=12)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Plot the time series data
plt.figure(figsize=(20, 10))
plt.plot(data.index, data['bike_count'], label='Bike Count')
plt.xlabel('Date')
plt.ylabel('Bike Count')
plt.title('Time Series of Bike Count')
plt.legend()
plt.show()

In [ ]:
# Split data into train-val-test
# ==============================================================================
data = data.loc['2012-01-01 00:00:00':'2014-12-30 23:00:00', :].copy()
end_train = '2013-12-31 23:59:00'
end_validation = '2014-11-30 23:59:00'
data_train = data.loc[: end_train, :].copy()
data_val   = data.loc[end_train:end_validation, :].copy()
data_test  = data.loc[end_validation:, :].copy()

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Validation dates : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

# Interactive plot of time series
# ==============================================================================
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['bike_count'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=data_val.index, y=data_val['bike_count'], mode='lines', name='Validation'))
fig.add_trace(go.Scatter(x=data_test.index, y=data_test['bike_count'], mode='lines', name='Test'))
fig.update_layout(
    title  = 'Hourly energy demand',
    xaxis_title="Time",
    yaxis_title="bike_count",
    legend_title="Partition:",
    width=750,
    height=370,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1, xanchor="left", x=0.001)
)
#fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
data['bike_count'].plot(kind='hist', bins=500, figsize=(20, 10))

In [ ]:
data_split = data[['bike_count']]
train = data_split.loc[data.index < '07-01-2024']
test = data_split.loc[data.index >= '07-01-2024']

fig, ax = plt.subplots(figsize=(20,10))
train.plot(ax=ax, label='Train split', title='Bike Count')
test.plot(ax=ax, label='Test split')
ax.axvline('07-01-2024', color='black', linestyle='--', lw=2)
plt.legend(['Train split', 'Test split'])

In [ ]:
data_split.loc[(data.index > '01-01-2024') & (data.index < '02-01-2024')].plot(figsize=(15,5), title="Month of data")

In [ ]:
data_split.loc[(data.index > '01-01-2024') & (data.index < '01-08-2024')].plot(figsize=(15,5), title="Week of data")


In [ ]:
tss = TimeSeriesSplit(n_splits=5, test_size=7, gap = 0)

fig, axs = plt.subplots(5,1, figsize=(15,15),sharex=True)

fold = 0
for train_index, test_index in tss.split(data_split):
    train = data_split.iloc[train_index]
    test = data_split.iloc[test_index]
    train.plot(ax=axs[fold], label=f'Train split {fold}', title='Bike Count')
    test.plot(ax=axs[fold], label=f'Test split {fold}')
    fold += 1
    
plt.tight_layout(rect=[0, 0, 1, 0.96])  
plt.legend(['Train split', 'Test split'])

In [ ]:
tss = ExpandingSlidingWindowSplitter(n_splits=10, fh=7, gap = 0)

fig, axs = plt.subplots(10,1, figsize=(15,15),sharex=True)

fold = 0
for train_index, test_index in tss.split(data_split):
    train = data_split.iloc[train_index]
    test = data_split.iloc[test_index]
    train.plot(ax=axs[fold], label=f'Train split {fold}', title='Bike Count')
    test.plot(ax=axs[fold], label=f'Test split {fold}')
    fold += 1
    
plt.tight_layout(rect=[0, 0, 1, 0.96])  
plt.legend(['Train split', 'Test split'])